In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from acquire import get_connection, new_zillow_data, get_zillow_data

In [2]:
df = get_zillow_data(cached=True)

In [3]:
longindex = df.loc[df['longitude'].isin(['NaN'])].index

In [4]:
 df.drop(longindex , inplace=True)

<h5>Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.</h5>

In [5]:

def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        null_count = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, null_count, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'null_count', 2 : '% of Total Values'})
        mz_table['Total Zeroes + Null Values'] = mz_table['Zero Values'] + mz_table['null_count']
        mz_table['% Total Zero + Null Values'] = 100 * mz_table['Total Zeroes + Null Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] >= 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " +  str((mz_table['null_count'] != 0).sum()) +
          " columns that have NULL values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_zero_values_table(df)

Your selected dataframe has 67 columns and 77380 Rows.
There are 52 columns that have NULL values.


,Zero Values,null_count,% of Total Values,Total Zeroes + Null Values,% Total Zero + Null Values,Data Type
buildingclasstypeid,0,77365,100.0,77365,100.0,float64
buildingclassdesc,0,77365,100.0,77365,100.0,object
finishedsquarefeet13,0,77339,99.9,77339,99.9,float64
storydesc,0,77330,99.9,77330,99.9,object
basementsqft,0,77330,99.9,77330,99.9,float64
storytypeid,0,77330,99.9,77330,99.9,float64
yardbuildingsqft26,0,77310,99.9,77310,99.9,float64
fireplaceflag,0,77208,99.8,77208,99.8,float64
architecturalstyletypeid,0,77174,99.7,77174,99.7,float64
architecturalstyledesc,0,77174,99.7,77174,99.7,object


<h4>Takeaways</h4>

Will need to look at individual columns to determine how to handle missing values. 

In [6]:
df2 = df.isnull().sum(axis =1).value_counts().sort_index(ascending=False)

In [7]:
type(df2)

pandas.core.series.Series

In [8]:
df2 = pd.DataFrame(df2)

In [9]:
df2.reset_index(level=0, inplace=True)

In [10]:
df2.columns

Index(['index', 0], dtype='object')

In [11]:
df2.columns= ['total_features_missing', 'total_rows_affected'] 

In [12]:
df2.head()

,total_features_missing,total_rows_affected
0,48,3
1,47,3
2,46,5
3,45,50
4,44,78


In [13]:
df2['pct_features_missing']= round((df2.total_features_missing /df.shape[1]) * 100, 2)

In [14]:
df2 = df2[['total_features_missing', 'pct_features_missing', 'total_rows_affected']]

In [16]:
df2.head(26)

,total_features_missing,pct_features_missing,total_rows_affected
0,48,71.64,3
1,47,70.15,3
2,46,68.66,5
3,45,67.16,50
4,44,65.67,78
5,43,64.18,28
6,42,62.69,23
7,41,61.19,29
8,40,59.70,230
9,39,58.21,285
